Backtest für die klassische Moving Average Crossover Strategie auf SPY angewendet;
Buy wenn avgshort_t-1 < avglong_t-1 und avgshort_t > avglong_t
Short wenn andersrum

Vorteile:
nutze momentum, bewährte Strategie

Nachteile:
lagging indicators, returns relativ gering, sehr bekannte strategie, daher eher schlecht

In [238]:
import numpy as np
import pandas as pd
import yfinance as yf
import scipy as sc
import plotly.graph_objects as plot

Regeln für diese Strategie:

Kaufe wenn:
    50DayEMA_t > 200Day_t
    50DayEMA_t-1 < 200Day_t-1 


Verkaufe wenn:
    50DayEMA_t < 200Day_t-1
    50DayEMA_t-1 > 200Day_t-1


Im folgenden Data vorbereiten und Moving Average für 50 und 200 Tage berechnen

In [239]:
ticker = yf.Ticker('BTC-EUR')
btcData = ticker.history(start = '2017-1-1', interval = '1d')
btcData.to_csv("./Data/btc_eur.csv")

In [240]:
btcData = pd.read_csv("./Data/YahooFinanceHistoricalPriceData/btc_eur.csv")
# calculate ema
# row 8
btcData["EWMA50Day"] = btcData["Close"].ewm(alpha = 1 / 20, adjust=False).mean()
# row 9
btcData["EWMA200Day"] = btcData["Close"].ewm(alpha = 1 / 100, adjust = False).mean()

ema = btcData.fillna(0)

# setze Start Werte
# row 10
btcData['Cash'] = 1000.0
maxRisk = 0.25
transactionCost = 0.001

# row 11
btcData['activePosition'] = 0.0

Zwei Möglichkeiten den Backtest durchzuführen:

1. Gehe iterativ durch den Dataframe durch und checke jede Reihe einzeln
    - gut für komplexe Strategien
    - mehr Aufwand
    - aber präzisere Durchführung, da Look Ahead Bias vermieden wird

2. Wende daten manipulation auf den gesamten Dataframe an
    - Funktioniert bei einfachen Strategien
    - Ist effizienter
    - weniger code benötigt

In [241]:
# gehe iterativ durch dataframe
for i in range(0, len(btcData)):

    # Long only Strategy
    #BUY
    if btcData.iloc[i]["EWMA50Day"] > btcData.iloc[i]["EWMA200Day"] and btcData.iloc[i-1]["EWMA50Day"] < btcData.iloc[i-1]["EWMA200Day"]:
        amountToBuy = btcData.iloc[i]['Cash'] * maxRisk 
        totalCost = amountToBuy + transactionCost * amountToBuy
        btcData.loc[i+1:len(btcData),'activePosition'] = amountToBuy / btcData.iloc[i+1]['Open']
        btcData.loc[i+1:len(btcData),'Cash'] = btcData.iloc[i]['Cash'] - totalCost
    #SELL
    if btcData.iloc[i]["EWMA50Day"] < btcData.iloc[i]["EWMA200Day"] and btcData.iloc[i-1]["EWMA50Day"] > btcData.iloc[i-1]["EWMA200Day"]:
        amountToSell = btcData.iloc[i]['activePosition'] * btcData.iloc[i+1]['Open']
        totalSell = amountToSell - transactionCost * amountToSell
        btcData.loc[i+1:len(btcData),'activePosition'] = 0
        btcData.loc[i+1:len(btcData),'Cash'] = btcData.iloc[i]['Cash'] + totalSell


    # Short only Strategy
    #BUY

    #SELL

btcData

TypeError: Slicing a positional slice with .loc is not allowed, Use .loc with labels or .iloc with positions instead.

In [236]:
# calculate Returns
btcData['Diff'] = btcData['Close'].diff()
btcData['Return'] = btcData['activePosition'] * btcData['Diff']
btcData['cumReturn'] = btcData['Return'].cumsum()

In [237]:
fig = plot.Figure([plot.Scatter(x=btcData.Date, y=btcData['cumReturn'], name='CumReturn'), 
                   plot.Scatter(x=btcData.Date, y=btcData['Close'], name='PriceHistory'), 
                   plot.Scatter(x=btcData.Date, y=btcData['Cash'], name='Cash')])
fig.show()

biggestPosition = btcData['activePosition'].max()
totalPercReturn = (btcData.iloc[-1]['activePosition'] * btcData.iloc[-1]['Close'] + btcData.iloc[-1]['Cash']) / btcData.iloc[0]['Cash']
percentageReturnHold = btcData.iloc[-1]['Close'] / btcData.iloc[0]['Close']

print(btcData.loc[btcData['activePosition'] == biggestPosition][['Date', 'activePosition']])
print(totalPercReturn)
print(percentageReturnHold)

                          Date  activePosition
51   2017-02-21 00:00:00+00:00        0.245696
52   2017-02-22 00:00:00+00:00        0.245696
53   2017-02-23 00:00:00+00:00        0.245696
54   2017-02-24 00:00:00+00:00        0.245696
55   2017-02-25 00:00:00+00:00        0.245696
..                         ...             ...
447  2018-03-24 00:00:00+00:00        0.245696
448  2018-03-25 00:00:00+00:00        0.245696
449  2018-03-26 00:00:00+00:00        0.245696
450  2018-03-27 00:00:00+00:00        0.245696
451  2018-03-28 00:00:00+00:00        0.245696

[401 rows x 2 columns]
4.910412414681137
102.63794706833187
